# Getting Started with Saul

We will look at a Spam Classification task where we try to classify email documents as SPAM or HAM. This notebook will work through the steps in creating and running the Spam Classifier.

### Step -1 - Jupyter-Scala integration for Saul

In [1]:
classpath.addRepository("http://cogcomp.cs.illinois.edu/m2repo")
classpath.add("edu.illinois.cs.cogcomp" %% "saul" % "0.5.5") 

34 new artifact(s)


34 new artifacts in macro
34 new artifacts in runtime
34 new artifacts in compile


### Step 0 - Spam Data

something about how the data looks like

In [2]:
import scala.io.Source
import java.io.File

val spamDataBasePath = "../../data/EmailSpam/"
val trainDataPath = spamDataBasePath  + "train/"
val testDataPath = spamDataBasePath  + "test/"

val dir = new File(trainDataPath + "ham")
require(dir.exists() && dir.isDirectory())

// val sampleDoc = dir.listFiles.filter(_.isFile).head
// $println("Sample Document:")
// Source.fromFile(sampleDoc).getLines.foreach(println(_))

import scala.io.Source
import java.io.File
spamDataBasePath: String = "../../data/EmailSpam/"
trainDataPath: String = "../../data/EmailSpam/train/"
testDataPath: String = "../../data/EmailSpam/test/"
dir: java.io.File = ../../data/EmailSpam/train/ham

### Step 1 - Data Reader

We create a reader that parses each document and parses it into required classes. 

We define an Email as a collection of words and its label.

In [3]:
case class Email(val words: Seq[String], val label: String)

object DataReader {
    def apply(dirName: String, label: String): Iterable[Email] = {
        val dir = new File(dirName)
        require(dir.exists() && dir.isDirectory)
        
        dir.listFiles
           .filter(_.isFile)
           .flatMap(file => parseEmail(file.getAbsolutePath, label))
    }
    
    private def parseEmail(fileName: String, label: String): Option[Email] = {
        val source = Source.fromFile(fileName)
        if (source.hasNext) {
            val words = source.getLines
                              .flatMap(_.split("\\s+"))
                              .toSeq
            Some(Email(words, label))
        } else {
            None
        }
    }
}


defined class Email
defined object DataReader

### Step 2 - DataModel (Entities, Features)

Where we define the DataModel

In [4]:
import edu.illinois.cs.cogcomp.saul.datamodel.DataModel

object SpamDataModel extends DataModel {
    val email = node[Email]
    
    // Features
    val words = property(email) { 
        doc: Email => doc.words.toList
    }
    
    val bigrams = property(email) {
        doc: Email => doc.words.sliding(2).map(_.mkString("-")).toList
    }
    
    val spamLabel = property(email) {
        doc: Email => doc.label
    }
}

import edu.illinois.cs.cogcomp.saul.datamodel.DataModel
defined object SpamDataModel

### Step 3 - Classifier

In [8]:
import edu.illinois.cs.cogcomp.lbjava.learn.SupportVectorMachine
import edu.illinois.cs.cogcomp.saul.classifier.Learnable
import SpamDataModel._

object SpamClassifier extends Learnable(email) {
    def label = spamLabel
    override lazy val classifier = new SupportVectorMachine()
    override def feature = using(words, bigrams)
}

import edu.illinois.cs.cogcomp.lbjava.learn.SupportVectorMachine
import edu.illinois.cs.cogcomp.saul.classifier.Learnable
import SpamDataModel._
defined object SpamClassifier

### Step 4 - App (Train, Test)

In [7]:
val trainData = DataReader(trainDataPath + "spam", "spam") ++ DataReader(trainDataPath + "ham", "ham")
val testData = DataReader(testDataPath + "spam", "spam") ++ DataReader(testDataPath + "ham", "ham")

SpamDataModel.email.populate(trainData)
SpamDataModel.email.populate(testData, train = false)

SpamClassifier.learn(30)
SpamClassifier.test()

INFO  [2016-11-06 18:30:11,121] cmd4$$user$SpamClassifier$: Learnable: Learn with data of size 9
INFO  [2016-11-06 18:30:11,122] cmd4$$user$SpamClassifier$: Training: 30 iterations remain.
INFO  [2016-11-06 18:30:11,123] cmd4$$user$SpamClassifier$: Training: 30 iterations remain.
INFO  [2016-11-06 18:30:11,248] cmd4$$user$SpamClassifier$: Training: 20 iterations remain.
INFO  [2016-11-06 18:30:11,348] cmd4$$user$SpamClassifier$: Training: 10 iterations remain.
 Label   Precision Recall    F1   LCount PCount
-----------------------------------------------
ham        100.000  60.000 75.000      5      3
spam        71.429 100.000 83.333      5      7
-----------------------------------------------
Accuracy    80.000    -      -      -        10


trainData: Iterable[Email] = ArraySeq(
  Email(
    Stream(
      "Subject:",
      "double",
      "your",
      "life",
      "insurance",
      "at",
      "no",
      "extra",
      "cost",
      "!",
      "29155",
      "the",
      "lowest",
      "life",
      "insurance",
      "quotes",
      "without",
...
testData: Iterable[Email] = ArraySeq(
  Email(
    Stream(
      "Subject:",
      "slotting",
      "order",
      "confirmation",
      "may",
      "18",
      ",",
      "2004",
      "etacitne",
      "{",
      "%",
      "begin",
      "_",
      "split",
      "76",
      "%",
      "}",
...
res6_5: edu.illinois.cs.cogcomp.saul.classifier.Results = Results(
  Array(
    ResultPerLabel(
      "ham",
      0.7499999999999999,
      1.0,
      0.6,
      Array("ham", "spam"),
      5,
      3,
      3
    ),
    ResultPerLabel(
      "spam",
      0.8333333333333333,
      0.7142857142857143,
      1.0,
      Array("ham", "spam"),
      5,
      7,
...